# scRFE source V2

In [39]:
# cleaning up code after meeting with Sevahn

In [40]:
# all code broken up into separate arguments for scRFE

In [41]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [42]:
# read in data 
adata = read_h5ad('/Users/madelinepark/Downloads/Limb_Muscle_facs.h5ad')
tiss = adata

In [43]:
# split data for training
def loc_split_function(tiss, feature='age'):
    tiss.obs['feature_type_of_interest'] = 'rest'
    results_feature_cv = pd.DataFrame()
    for c in list(set(tiss.obs[feature])): 
        feature_of_interest = c
        tiss.obs.loc[tiss.obs[tiss.obs[feature] == feature_of_interest].index,'feature_type_of_interest'] = feature_of_interest
        feat_labels = tiss.var_names 
        X = tiss.X
        y = tiss.obs['feature_type_of_interest']
    return X, y, feature, feat_labels #this is returning only the last thing in the loop 

In [44]:
# create random forest and selector, then train
def train(X, y, test_size, random_state):
    print('training...')
    loc_split = loc_split_function(tiss=tiss, feature='age')
    X = loc_split[0]
    y = loc_split[1]
    feat_labels = loc_split[3] #this should not be hardcoded #genes be argument
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4)
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_]
    return selector, clf, feat_labels, feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test

In [47]:
cd /Users/madelinepark/src3/scRFE/scRFE-results

/Users/madelinepark/src3/scRFE/scRFE-results


In [48]:
# result write
def result_write (c, feature_selected, tiss, feature='age',test_size=0.05, random_state=0): 
    results_feature_cv = pd.DataFrame()
    print('result writing...')
    X = loc_split(tiss=tiss, feature= feature)[0]
    y = loc_split(tiss=tiss, feature= feature)[1]
    feat_labels = train(X, y, test_size= 0.05, random_state=0)[0] 
    feature_selected = train(X, y, test_size, random_state)[3]
    gini_scores = train(X, y, test_size, random_state)[4]
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected 
    resaux[c + '_gini'] = (gini_scores) 
#     print(feature_selected)
#     print (gini_scores)
    results_feature_cv = pd.concat([results_feature_cv,resaux],axis=1)
    tiss.obs['feature_type_of_interest'] = 'rest'
    print(results_feature_cv)
    results_feature_cv.to_csv('scRFE-results.csv') #trying to save results as a csv
    return results_feature_cv

In [49]:
# combined functions
def scRFE (tiss=tiss, X=tiss.X, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
    for c in list(set(tiss.obs[feature])): 
        print(c)
        X = loc_split(tiss=tiss, feature= feature)[0] #change age to feature
        y = loc_split(tiss=tiss, feature= feature)[1]
        feature = loc_split(tiss=tiss, feature= feature)[2]
        feat_labels = loc_split(tiss=tiss, feature= feature)[3]
        feature_selected = train(X, y, test_size, random_state)[1]
        X_train = train(X, y, test_size, random_state)[2]
        X_test = train(X, y, test_size, random_state)[3]
        y_train = train(X, y, test_size, random_state)[4]
        y_test = train(X, y, test_size, random_state)[5]
        result_write(c, feature_selected, tiss, feature=feature,test_size=0.05, random_state=0)
        tiss.obs['age_type_of_interest'] = 'rest'
#         not sorted

In [50]:
# run function
scRFE(tiss=tiss, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)

24m
training...
training...
training...
training...
training...
result writing...
training...
training...
training...
                24m  24m_gini
0     0610007C21Rik  0.000098
1     0610007L01Rik  0.000057
2     0610007N19Rik  0.000034
3     0610007P22Rik  0.000014
4     0610009D07Rik  0.000036
5     0610009O20Rik  0.000029
6     0610010K14Rik  0.000082
7     0610010O12Rik  0.000025
8     0610011F06Rik  0.000028
9     0610031J06Rik  0.000032
10    0610037L13Rik  0.000057
11    0910001L09Rik  0.000041
12    1110002B05Rik  0.000046
13    1110003E01Rik  0.000169
14    1110004F10Rik  0.000055
15    1110007C09Rik  0.000068
16    1110008F13Rik  0.000070
17    1110037F02Rik  0.000035
18    1110038B12Rik  0.000010
19    1110038F14Rik  0.000048
20    1110051M20Rik  0.000026
21    1190002H23Rik  0.000081
22    1190002N15Rik  0.000070
23    1190003J15Rik  0.000058
24    1300014I06Rik  0.000058
25    1500003O03Rik  0.001863
26    1500012F01Rik  0.000054
27    1500032L24Rik  0.000087
28    160002

In [30]:
results_feature_cv

NameError: name 'results_feature_cv' is not defined

In [ ]:
# # combined functions
# def scRFE_save (tiss=tiss, X=tiss.X, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
#     for c in list(set(tiss.obs[feature])): 
#         print(c)
#         X = loc_split(tiss=tiss, feature= feature)[0] #change age to feature
#         y = loc_split(tiss=tiss, feature= feature)[1]
#         feature = loc_split(tiss=tiss, feature= feature)[2]
#         feat_labels = loc_split(tiss=tiss, feature= feature)[3]
#         feature_selected = train(X, y, test_size, random_state)[1]
#         X_train = train(X, y, test_size, random_state)[2]
#         X_test = train(X, y, test_size, random_state)[3]
#         y_train = train(X, y, test_size, random_state)[4]
#         y_test = train(X, y, test_size, random_state)[5]
#         result_write(c, feature_selected, tiss, feature=feature,test_size=0.05, random_state=0)
#         tiss.obs['age_type_of_interest'] = 'rest'
# #         not sorted